# Initialization 

In [1]:
import numpy as np 
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler 
import tensorflow as tf

# Checking the tensorflow version

In [2]:
print(tf.__version__)

1.13.1


In [3]:
x = tf.Variable(5,name="X")
y = tf.Variable(7,name="Y")
f=(x*x*y) + y +2
f2=(x*x*y)
result =None


Instructions for updating:
Colocations handled automatically by placer.


# Method 0
## Using Initializer 


In [4]:
sess = tf.Session()
sess.run(x.initializer)

print('X => ', x.value())

sess.run(y.initializer)

print('Y => ',y.value())
result = sess.run(f)

print('result => ',result)

X =>  Tensor("X/read:0", shape=(), dtype=int32)
Y =>  Tensor("Y/read:0", shape=(), dtype=int32)
result =>  184


# Method 1
## Intializing the Session

In [5]:
with tf.Session() as sess:
    x.initializer.run() #tf.get_default_session.run(x.initializer)
    print('x => ',x.eval())
    y.initializer.run()
    print('y => ',y.eval())
    result=f.eval()     #tf.get_default_session.run(f)
    print('eval => ',f.eval())
 

x =>  5
y =>  7
eval =>  184


# Method 2
## Using Global Variables Initializer

In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # handles all the initialisation of x , y
    result = f.eval()
    print('result ',result)

result  184


# Method 3
## InteractiveSession

In [7]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result1,result2 = sess.run([f,f2]) #(x*x*y) will be reused
sess.close()


x1=  tf.Variable(2)
same_graph = x1.graph is tf.get_default_graph()

print('is same graph = ',same_graph)

x2=None
graph_1 = tf.Graph()
with graph_1.as_default():
    x2=tf.Variable(2)

same_graph  = x2.graph is tf.get_default_graph()

print('is same graph = ',same_graph)

same_graph  = x2.graph is graph_1

print('is same graph = ',same_graph)

print(result1)
print(result2)

is same graph =  True
is same graph =  False
is same graph =  True
184
175


# Performing linear Regression - Normal Equations

$$ \theta = (X^T . X )^{-1} . X^T.Y  $$

In [8]:
housing = fetch_california_housing()

m , n = housing.data.shape

data_bias = np.c_[np.ones((m,1)),housing.data]

X = tf.constant(data_bias , dtype=tf.float32 , name='X')
Y = tf.constant(housing.target.reshape(-1,1) , dtype=tf.float32 , name='Y')

X_T = tf.transpose(X)
theta = tf.matmul( tf.matmul( tf.matrix_inverse( tf.matmul(X_T ,X) ) , X_T ) , Y )

with tf.Session() as sess:
    result = sess.run(theta)
    print('result => ',result)

    

result =>  [[-3.6959320e+01]
 [ 4.3698898e-01]
 [ 9.4245886e-03]
 [-1.0791138e-01]
 [ 6.4842808e-01]
 [-3.9986235e-06]
 [-3.7866351e-03]
 [-4.2142656e-01]
 [-4.3467718e-01]]


# Linear Regression - Batch Gradient Descent

In [9]:
housing_data = fetch_california_housing()

m_,n=housing_data.data.shape

housing_data_plus_bias = np.c_[np.ones((m_,1)),housing_data.data]

scalar = StandardScaler()

# scalar.fit(housing_data_plus_bias)

scaled_housing_data_plus_bias = scalar.fit_transform(housing_data_plus_bias)

scaled_housing_data = scalar.fit_transform(housing_data.data)

# print(scaled_housing_data_plus_bias)

# print(housing_data_plus_bias)

# print(housing_data_plus_bias.shape)

learning_rate=0.01
epochs = 1000

x = tf.constant(scaled_housing_data , dtype=tf.float32 , name='x')

y = tf.constant(housing_data.target.reshape(-1,1) , dtype=tf.float32 , name='y')

x_t = tf.transpose(x)
# theta = tf.matmul(tf.matrix_inverse(tf.matmul(x_t,x)),tf.matmul(x_t,y))

# theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")

m = tf.Variable(tf.random_uniform([n,1],-1.0,1.0),name="m")
b = tf.Variable(tf.random_uniform([1,1],-1.0,1.0),name="b")

y_pred= tf.matmul(x,m)+b

error = tf.square(y_pred - y)
mse=tf.reduce_mean(error,name='mse')

#y_ = (tf.matmul(x,m)+b)

m_gradients = tf.Variable(-2/m_ * tf.matmul(x_t,y-y_pred))

b_gradients = tf.Variable((-2/m_) * tf.reduce_mean(y-y_pred))

# m_gradients_upd = tf.assign(m_gradients,m_gradients)

# b_gradients_upd = tf.assign(b_gradients,b_gradients)


m_ops = tf.assign(m, m- (learning_rate * m_gradients))
b_ops = tf.assign(b, b- (learning_rate * b_gradients))


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(epochs):
        if i % 100 == 0 :
            print("MSE = {}".format(mse.eval()))
        sess.run([m_ops,b_ops])
    
    print("Reduced MSE = {}".format(mse.eval()))
    print("Best m = {} ,b = {}".format(m.eval(),b.eval()))

MSE = 8.728497505187988
MSE = 38.081382751464844
MSE = 200.0938262939453
MSE = 494.7647705078125
MSE = 922.0956420898438
MSE = 1482.0877685546875
MSE = 2174.7392578125
MSE = 3000.04931640625
MSE = 3958.020263671875
MSE = 5048.6513671875
Reduced MSE = 6271.9208984375
Best m = [[ 25.470697 ]
 [ -9.938125 ]
 [ -2.8303971]
 [-10.015552 ]
 [ 18.126503 ]
 [ 10.552018 ]
 [-35.946396 ]
 [ 30.530895 ]] ,b = [[0.47862816]]


# Linear Regression - using tensorflow Autodiff()

In [10]:
m_,n=housing_data.data.shape

scalar = StandardScaler()

scaled_housing_data = scalar.fit_transform(housing_data.data)

learning_rate=0.01
epochs = 1000

x = tf.constant(scaled_housing_data ,dtype=tf.float32,name='x')
y = tf.constant(housing_data.target.reshape(-1,1) ,dtype=tf.float32,name='y')
x_t = tf.transpose(x)

m = tf.Variable(tf.random_uniform([n,1],-1.0,1.0),name="m")
b = tf.Variable(tf.random_uniform([1,1],-1.0,1.0),name="b")

y_pred= tf.matmul(x,m)+b

error = tf.square(y_pred - y)
mse=tf.reduce_mean(error,name='mse')

m_gradients = tf.gradients(mse , [m])[0]

b_gradients = tf.gradients(mse , [b])[0]

m_ops = tf.assign(m, m- (learning_rate * m_gradients))
b_ops = tf.assign(b, b- (learning_rate * b_gradients))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(epochs):
        if i % 100 == 0 :
            print("MSE = {}".format(mse.eval()))
        sess.run([m_ops,b_ops])
    
    print("Reduced MSE = {}".format(mse.eval()))
    print("Best m = {} ,b = {}".format(m.eval(),b.eval()))

MSE = 6.875275611877441
MSE = 0.6984758377075195
MSE = 0.5796214938163757
MSE = 0.5644892454147339
MSE = 0.5564892292022705
MSE = 0.5503751039505005
MSE = 0.5455024838447571
MSE = 0.5415905714035034
MSE = 0.538436770439148
MSE = 0.5358853340148926
Reduced MSE = 0.5338143110275269
Best m = [[ 0.916736  ]
 [ 0.14679849]
 [-0.40821066]
 [ 0.41404727]
 [ 0.0045683 ]
 [-0.04355609]
 [-0.6136149 ]
 [-0.5933496 ]] ,b = [[2.0685523]]


# Linear Regression - Using Optimizers

    - GradientDescent Optimizer
    - Momentem Optimizer

In [14]:
m_,n=housing_data.data.shape

scalar = StandardScaler()

scaled_housing_data = scalar.fit_transform(housing_data.data)

learning_rate=0.01
epochs = 1000

x = tf.constant(scaled_housing_data ,dtype=tf.float32,name='x')
y = tf.constant(housing_data.target.reshape(-1,1) ,dtype=tf.float32,name='y')
x_t = tf.transpose(x)

m = tf.Variable(tf.random_uniform([n,1],-1.0,1.0),name="m")
b = tf.Variable(tf.random_uniform([1,1],-1.0,1.0),name="b")

y_pred= tf.matmul(x,m)+b

error = tf.square(y_pred - y)

# Optimiser 

# learning_rate = 0.01;
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

mse = tf.reduce_mean(error,name='mse')

'''
m_gradients = tf.gradients(mse , [m])[0]

b_gradients = tf.gradients(mse , [b])[0]

m_ops = tf.assign(m, m- (learning_rate * m_gradients))

b_ops = tf.assign(b, b- (learning_rate * b_gradients))

'''

m_ops = optimizer.minimize(mse,var_list=[m]) 

b_ops = optimizer.minimize(mse,var_list=[b]) 


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(epochs):
        if i % 100 == 0 :
            print("MSE = {}".format(mse.eval()))
        sess.run([m_ops,b_ops])
    
    print("Reduced MSE = {}".format(mse.eval()))
    print("Best m = {} ,b = {}".format(m.eval(),b.eval()))

MSE = 7.3643670082092285
MSE = 0.7188141345977783
MSE = 0.6192888021469116
MSE = 0.5912034511566162
MSE = 0.5724892616271973
MSE = 0.5590720176696777
MSE = 0.5493972301483154
MSE = 0.5424172282218933
MSE = 0.5373811721801758
MSE = 0.5337474346160889
Reduced MSE = 0.5311253070831299
Best m = [[ 0.7877081 ]
 [ 0.14335231]
 [-0.12563138]
 [ 0.16331516]
 [ 0.00498985]
 [-0.04051354]
 [-0.7599041 ]
 [-0.72274435]] ,b = [[2.0685523]]


In [15]:
m_,n=housing_data.data.shape

scalar = StandardScaler()

scaled_housing_data = scalar.fit_transform(housing_data.data)

learning_rate=0.01
epochs = 1000

x = tf.constant(scaled_housing_data ,dtype=tf.float32,name='x')
y = tf.constant(housing_data.target.reshape(-1,1) ,dtype=tf.float32,name='y')
x_t = tf.transpose(x)

m = tf.Variable(tf.random_uniform([n,1],-1.0,1.0),name="m")
b = tf.Variable(tf.random_uniform([1,1],-1.0,1.0),name="b")

y_pred= tf.matmul(x,m)+b

error = tf.square(y_pred - y)

# Optimiser 

# learning_rate = 0.01;
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate,momentum=0.9)

mse = tf.reduce_mean(error,name='mse')

'''
m_gradients = tf.gradients(mse , [m])[0]

b_gradients = tf.gradients(mse , [b])[0]

m_ops = tf.assign(m, m- (learning_rate * m_gradients))

b_ops = tf.assign(b, b- (learning_rate * b_gradients))

'''

m_ops = optimizer.minimize(mse,var_list=[m]) 

b_ops = optimizer.minimize(mse,var_list=[b]) 


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(epochs):
        if i % 100 == 0 :
            print("MSE = {}".format(mse.eval()))
        sess.run([m_ops,b_ops])
    
    print("Reduced MSE = {}".format(mse.eval()))
    print("Best m = {} ,b = {}".format(m.eval(),b.eval()))

MSE = 12.922515869140625
MSE = 0.5289731621742249
MSE = 0.5244243741035461
MSE = 0.5243261456489563
MSE = 0.5243215560913086
MSE = 0.5243210196495056
MSE = 0.5243210196495056
MSE = 0.5243210196495056
MSE = 0.5243210196495056
MSE = 0.5243210196495056
Reduced MSE = 0.5243210196495056
Best m = [[ 0.8296215 ]
 [ 0.11875202]
 [-0.265531  ]
 [ 0.30569962]
 [-0.00450288]
 [-0.03932635]
 [-0.8998811 ]
 [-0.87053674]] ,b = [[2.068558]]


# Mini-batch Gradient Descent

In [12]:
m_,n=housing_data.data.shape

scalar = StandardScaler()

scaled_housing_data = scalar.fit_transform(housing_data.data)

learning_rate=0.01
epochs = 1000

x = tf.constant(scaled_housing_data ,dtype=tf.float32,name='x')
y = tf.constant(housing_data.target.reshape(-1,1) ,dtype=tf.float32,name='y')
x_t = tf.transpose(x)

m = tf.Variable(tf.random_uniform([n,1],-1.0,1.0),name="m")
b = tf.Variable(tf.random_uniform([1,1],-1.0,1.0),name="b")

y_pred= tf.matmul(x,m)+b

error = tf.square(y_pred - y)
mse=tf.reduce_mean(error,name='mse')


m_gradients = tf.gradients(mse , [m])[0]

b_gradients = tf.gradients(mse , [b])[0]

m_ops = tf.assign(m, m- (learning_rate * m_gradients))
b_ops = tf.assign(b, b- (learning_rate * b_gradients))


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(epochs):
        if i % 100 == 0 :
            print("MSE = {}".format(mse.eval()))
        sess.run([m_ops,b_ops])
    
    print("Reduced MSE = {}".format(mse.eval()))
    print("Best m = {} ,b = {}".format(m.eval(),b.eval()))

MSE = 9.870588302612305
MSE = 0.7254812121391296
MSE = 0.5643523931503296
MSE = 0.5519969463348389
MSE = 0.54473876953125
MSE = 0.5394521951675415
MSE = 0.5355719327926636
MSE = 0.5327169895172119
MSE = 0.5306108593940735
MSE = 0.5290526151657104
Reduced MSE = 0.5278964042663574
Best m = [[ 0.76135683]
 [ 0.12525451]
 [-0.10045449]
 [ 0.15331931]
 [-0.00125129]
 [-0.03842087]
 [-0.9165835 ]
 [-0.8775474 ]] ,b = [[2.0685523]]
